In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import json

In [ ]:
%run GetLossObj.ipynb

In [ ]:
baseDir = "../save/Final/Traffic/RNN/server1-model23/"
# expData = torch.load(baseDir+"experimentData.pth")
plotTrainReconLosses = torch.load(baseDir + "plot_train_recon_losses", map_location="cpu")
plotValReconLosses = torch.load(baseDir + "plot_val_recon_losses", map_location="cpu")
with open(baseDir + "args.txt") as f:
    args = f.read()
args = Bunch(json.loads(args))
args.no_cuda = True
args.cuda = False
args.dropout= 0.0
if args.dataset == "traffic":
    args.output_dim = args.x_dim
else: #args.dataset == "human":
    args.output_dim = args.x_dim * 2
args.input_sequence_len = args.sequence_len
args.target_sequence_len = args.sequence_len

In [ ]:
print(args)

In [ ]:
print(args.initial_lr, args.dataset)

In [ ]:
dataDict = getDataLoaders("/Users/danielzeiberg/Data/Traffic/Processed/trafficWithTime/")


In [ ]:
getlossobj = PostProcess(baseDir,args, dataDict=dataDict, chooseModel="rnn")

In [ ]:
print("min val loss: {:.4f} at idx {}".format(np.min(plotValReconLosses), np.argmin(plotValReconLosses)))

In [ ]:
valRetVals = getlossobj.getLossAtEpoch("rnn_full_model.pth", "val")

In [ ]:
trainRetVals = getlossobj.getLossAtEpoch("rnn_full_model.pth", "train")

In [ ]:
testRetVals = getlossobj.getLossAtEpoch("rnn_full_model.pth", "test")

In [ ]:
for k, v in testRetVals["reconLosses"].items():
    print(5 + 5*k, v)

In [ ]:
print("train: {:.4f}, val: {:.4f}, test: {:.4f}".format(trainRetVals["reconLoss"],valRetVals["reconLoss"], testRetVals["reconLoss"]))

In [ ]:
def plotRandomInstance(preds,targets, datas, dataset):
    instance = np.random.randint(datas.shape[0])
    sensor = np.random.randint(datas.shape[2])
    p = preds[instance, :, sensor]
    t = targets[instance, :, sensor]
    d = datas[instance, :, sensor]
    plt.plot(range(5,65, 5),p, label="Pred")
    #plt.fill_between(range(0,60, 5),m-1.96*std, m+1.96*std)
    plt.plot(range(5,65, 5),t, label="target")
    plt.plot(range(-55,5, 5), d, label="input")
    plt.legend()
    plt.xlabel("Minutes relative to prediction time")
    plt.ylabel("Speed (mph)")
    plt.title("random sensor sample {}".format(dataset))
    plt.show()

In [ ]:
plotRandomInstance(trainRetVals["preds"], trainRetVals["targets"], trainRetVals["datas"], "Train")

In [ ]:
plotRandomInstance(valRetVals["preds"], valRetVals["targets"], valRetVals["datas"], "Validation")

In [ ]:
plotRandomInstance(testRetVals["preds"], testRetVals["targets"], testRetVals["datas"], "Test")

In [ ]:
total = 0
for p in getlossObj.model.parameters():
    total += np.prod(p.shape)
print(total)

In [ ]:
print(getlossObj.dataDict["train_mean"], getlossObj.dataDict["val_mean"], getlossObj.dataDict["train_std"], getlossObj.dataDict["val_std"])

In [ ]:
plt.plot(np.sqrt(np.mean((predsVal - targetsVal)**2, axis=(1,2))))
ticks = plt.xticks(np.arange(0,12), np.arange(5,13*5, 5))
plt.xlabel("Minutes after prediction time")
plt.ylabel("RMSE")


In [ ]:
plotTrainValCurve(plotTrainReconLosses, plotValReconLosses)

In [ ]:
next(iter(getlossobj.model.parameters())).is_cuda

In [ ]:
for name, p in getlossobj.model.named_parameters():
    if bias in 
    print(name)